# CoreNLP Stanza

Esta herramienta de la Universidad de Stanford nos permite tokenizar el texto, etiquetar las partes del discurso y buscar entidades nombradas. Su documentación puede encontrarse en https://stanfordnlp.github.io/stanza/index.html .

In [1]:
import stanza
from pymongo import MongoClient, UpdateOne
from pprint import pprint
import re

## Descarga y configuración de Stanza

In [3]:
stanza.download('es', processors='tokenize,mwt,pos,ner')

2020-05-22 16:00:00 INFO: Downloading these customized packages for language: es (Spanish)...
| Processor       | Package  |
------------------------------
| tokenize        | ancora   |
| mwt             | ancora   |
| pos             | ancora   |
| ner             | conll02  |
| pretrain        | ancora   |
| backward_charlm | newswiki |
| forward_charlm  | newswiki |

2020-05-22 18:13:31 INFO: Finished downloading models and saved to C:\Users\Usuario.000\stanza_resources.


In [3]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,ner', use_gpu=True)

2020-05-23 11:36:14 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| ner       | conll02 |

2020-05-23 11:36:14 INFO: Use device: cpu
2020-05-23 11:36:14 INFO: Loading: tokenize
2020-05-23 11:36:14 INFO: Loading: mwt
2020-05-23 11:36:14 INFO: Loading: pos
2020-05-23 11:36:16 INFO: Loading: ner
2020-05-23 11:36:20 INFO: Done loading processors!


## Conexión a Base de Datos

In [2]:
uri = open('./../../mongo_uri.txt', 'r', encoding='utf-8').read()
client = MongoClient(uri)
print(client)
db = client.boletines_db

MongoClient(host=['clustershido-shard-00-02-uxkjr.gcp.mongodb.net:27017', 'clustershido-shard-00-01-uxkjr.gcp.mongodb.net:27017', 'clustershido-shard-00-00-uxkjr.gcp.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='ClusterShido-shard-0', ssl=True)


### Obtener todos los boletines

In [37]:
cursor = db.boletines.find(
    {}, 
    {'boletin':1, 'encabezado':1, 'sumarios':1}
).sort('fecha')

## Tokenize, POS y NER

In [38]:
# Agrego punto final a oraciones que no tienen
patron_p = re.compile(r'([^\.])(\n|$)')
remplazo_p = r'\1.\n'

patron_com = re.compile(r'[\u201c\u201d]')
remplazo_com = r'"'

In [39]:
def procesar_sent(txt, nlp):
    doc = nlp(txt)
    return [
        { 
            'tokens' : [
                token.to_dict() for token in sent.tokens
            ],
            'entities' : [
                ent.to_dict() for ent in sent.entities
            ],
        }
        for sent in doc.sentences
    ]

In [40]:
%%time
updates = []
i=0

for boletin in cursor:
    filtro = {'boletin_id': boletin['_id']}
    update = {'$set' : {}}
    
    # Encabezado
    txt = patron_p.sub(remplazo_p, boletin['encabezado'])
    txt = patron_com.sub(remplazo_com, txt)
    # Procesar oración
    update['$set']['encabezado'] = procesar_sent(txt, nlp)
    
    # Sumarios
    try:
        txt = patron_p.sub(remplazo_p, '\n'.join(boletin['sumarios']))
        txt = patron_com.sub(remplazo_com, txt)
        sentences_sum = procesar_sent(txt, nlp)
        update['$set']['sumarios'] = sentences_sum
    except KeyError:
        sentences_sum = []
    
    # Contenido principal
    txt = patron_com.sub(remplazo_com, boletin['boletin'])
    # Procesar oración
    update['$set']['boletin'] = procesar_sent(txt, nlp)
    
    # Cada 35 iteraciones, imprimimos resultados
    if i%35==0:
        l_sent = [
            update['$set']['encabezado'], 
            sentences_sum, 
            update['$set']['boletin']
        ]
        orden = ['Encabezado', 'Sumarios', 'Boletín']
        for titulo, sentences in zip(orden, l_sent):
            print(f'======= Oraciones en {titulo} ======')
            for i_sent, sentence in enumerate(sentences):
                print(f'==== Oración {i_sent+1} ====')
                
                tokens = [token[0]['text'] for token in sentence['tokens']]
                # tokens = sentence['tokens']
                print('-- Tokens:')
                # print(tokens)
                print(' '.join(tokens))
                
                nes = [ent['text'] for ent in sentence['entities']]
                # nes = sentence['entities']
                if len(nes) > 0:
                    print('-- Named Entities:')
                    # print(nes)
                    print('\n'.join(nes))
        print('========================= FIN =======================\n\n')
    
    updates.append(UpdateOne(filtro, update, upsert = True)) 
    i+=1
cursor.rewind()

======= Oraciones en Encabezado ======
==== Oración 1 ====
-- Tokens:
José Antonio Meade presentó la Coordinación de Atención Ciudadana y Buzón del Candidato .
-- Named Entities:
José Antonio Meade
Coordinación de Atención Ciudadana
Buzón del Candidato
======= Oraciones en Sumarios ======
==== Oración 1 ====
-- Tokens:
El abanderado presidencial refrendó su compromiso de encabezar un gobierno a la medida de cada persona .
======= Oraciones en Boletín ======
==== Oración 1 ====
-- Tokens:
El candidato de la coalición Todos por México a la Presidencia de la República , José Antonio Meade , presentó la Coordinación de Atención Ciudadana y el Buzón del Candidato .
-- Named Entities:
Todos por México
Presidencia de la República
José Antonio Meade
Coordinación de Atención Ciudadana
Buzón del Candidato
==== Oración 2 ====
-- Tokens:
A través de estos mecanismos , Meade y su esposa , Juana Cuevas , recabarán las propuestas , opiniones , inquietudes y necesidades de la población .
-- Named Enti

======= Oraciones en Encabezado ======
==== Oración 1 ====
-- Tokens:
José Antonio Meade se comprometió a construir un mejor destino para Coahuila .
-- Named Entities:
José Antonio Meade
Coahuila
======= Oraciones en Sumarios ======
======= Oraciones en Boletín ======
==== Oración 1 ====
-- Tokens:
El candidato de la coalición Todos por México , José Antonio Meade , se comprometió a forjar un mejor destino para Coahuila , al desarrollo de la infraestructura en el estado y a hacer equipo con las mujeres y los jóvenes para ganar la elección del próximo primero de julio .
-- Named Entities:
Todos por México
José Antonio Meade
Coahuila
==== Oración 2 ====
-- Tokens:
En un evento masivo en Saltillo , ante más de 10 mil simpatizantes que gritaban " Pepe presidente , Pepe presidente " , el aspirante del PRI , PVEM y Nueva Alianza dijo que en Coahuila son invencibles y anunció que , como presidente , será el mejor aliado del gobernador Miguel Riquelme para avanzar en el desarrollo de la entida

======= Oraciones en Encabezado ======
==== Oración 1 ====
-- Tokens:
Observa AMLO que Anaya le fue peor que a Meade y " nosotros para arriba como la espuma del mar " .
-- Named Entities:
AMLO
Anaya
Meade
======= Oraciones en Sumarios ======
==== Oración 1 ====
-- Tokens:
Invita a Alicia Bárcena para que cuando concluya su función en la CAPAL sea la embajadora en la Organización de las Naciones Unidas .
-- Named Entities:
Alicia Bárcena
CAPAL
Organización de las Naciones Unidas
==== Oración 2 ====
-- Tokens:
Agradece a la cantante Belinda su apoyo y su solidaridad , es una extraordinaria artista .
-- Named Entities:
Belinda
==== Oración 3 ====
-- Tokens:
Convoca AMLO a dirigentes del MORENA , PT y PES para enseñar a ciudadanos cómo votar el próximo 1 de julio .
-- Named Entities:
AMLO
MORENA
PT
PES
======= Oraciones en Boletín ======
==== Oración 1 ====
-- Tokens:
Andrés Manuel López Obrador observó que en el segundo debate le fue peor a Anaya que a Meade , porque Anaya quedó al descub

======= Oraciones en Encabezado ======
==== Oración 1 ====
-- Tokens:
Convoca AMLO a reforzar el trabajo de defensa del voto y tener representantes en todas las casillas .
-- Named Entities:
AMLO
======= Oraciones en Sumarios ======
==== Oración 1 ====
-- Tokens:
Pide no confiarse , porque se enfrente a una mafia del poder , es gente que no tiene escrúpulos morales , hacen fraudes electorales .
======= Oraciones en Boletín ======
==== Oración 1 ====
-- Tokens:
El candidato por la coalición " Juntos haremos historia " por la Presidencia de México , Andrés Manuel López Obrador convocó a quienes trabajan en la defensa de los votos a reforzar el trabajo de organización y tener representantes en todas las casillas .
-- Named Entities:
Presidencia de México
Andrés Manuel López Obrador
==== Oración 2 ====
-- Tokens:
Pidió no confiarse , porque se enfrente a una mafia del poder , es gente que no tiene escrúpulos morales , hacen fraudes electorales , no se respeta la voluntad de los ciudadanos 

Wall time: 34min 52s


In [41]:
resw = db.stats_corenlp.bulk_write(updates)
pprint(resw.bulk_api_result)

{'nInserted': 0,
 'nMatched': 2,
 'nModified': 0,
 'nRemoved': 0,
 'nUpserted': 345,
 'upserted': [{'_id': ObjectId('5ec9718807772823d561eb76'), 'index': 2},
              {'_id': ObjectId('5ec9718807772823d561eb77'), 'index': 3},
              {'_id': ObjectId('5ec9718807772823d561eb78'), 'index': 4},
              {'_id': ObjectId('5ec9718807772823d561eb79'), 'index': 5},
              {'_id': ObjectId('5ec9718807772823d561eb7c'), 'index': 6},
              {'_id': ObjectId('5ec9718807772823d561eb80'), 'index': 7},
              {'_id': ObjectId('5ec9718807772823d561eb82'), 'index': 8},
              {'_id': ObjectId('5ec9718807772823d561eb85'), 'index': 9},
              {'_id': ObjectId('5ec9718807772823d561eb87'), 'index': 10},
              {'_id': ObjectId('5ec9718807772823d561eb90'), 'index': 11},
              {'_id': ObjectId('5ec9718807772823d561eb96'), 'index': 12},
              {'_id': ObjectId('5ec9718807772823d561eb99'), 'index': 13},
              {'_id': ObjectId('5ec

In [42]:
client.close()